In [1]:
from lavis_lib import *

In [15]:
# a sample:

adapted_rootpath = os.path.join('..',ROOT_OUTFOLDERPATH)
img_path = choice(get_all_images(adapted_rootpath))
image = Image.open(img_path).convert("RGB")

plt.figure(figsize=(20,20))
plt.imshow(image)
plt.axis('off')
plt.show()

IndexError: list index out of range

In [19]:
os.listdir(ROOTPATH)

['outputs']